In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import numpy as np
import scipy as sp
import sklearn as sl
from scipy import stats
from sklearn import datasets
from sklearn import linear_model

In [9]:
IDnumber = 1242830  #YOUR_ID , try also to change the seed to see the impact of random initialization on the results
np.random.seed(IDnumber)

In [10]:
#load the dataset
filename = 'NBA.csv'
NBA = csv.reader(open(filename, newline=''), delimiter=',')

header = next(NBA) #skip first line
print(header)

dataset = list(NBA)
for i in range(len(dataset)):
    dataset[i] = [int(x) for x in dataset[i]]
    
dataset = np.asarray(dataset)

X = dataset[:,1:4] #columns 1,2,3 contain the features
Y = dataset[:,0]  # column 0: labels

Y = Y*2-1  # set labels to -1, 1 as required by perceptron implementation

m = dataset.shape[0]
print("The length of the dataset is",m,"rows")
permutation = np.random.permutation(m) # random permurtation

X = X[permutation]
Y = Y[permutation]

['Position', 'Height cm', 'kg', 'Age']
The length of the dataset is 260 rows


In [11]:
#Divide in training and test: make sure that your training set
#contains at least 10 elements from class 1 and at least 10 elements
#from class -1! If it does not, modify the code so to apply more random
#permutations (or the same permutation multiple times) until this happens.
#IMPORTANT: do not change the random seed.

#m_training needs to be the number of samples in the training set
m_training = int((len(X)*70)/100) # m_training must be an integer number
                                  # equal to the 70% of the data

#m_test needs to be the number of samples in the test set
m_test = int((len(X)*30)/100)  # m_test must be an integer number
                               # equal to the 30% of the data

#X_training = instances for training set
X_training = X[:m_training]
#Y_training = labels for the training set
Y_training = Y[:m_training]

if (np.count_nonzero(Y_training == 1) >= 10) & (np.count_nonzero(Y_training == -1) >= 10):
    print("The training set contains at least 10 elements of class 1 AND at least 10 elements of class -1")
else: 
    print("The training set needs permutations")
    while((np.count_nonzero(Y_training == 1) < 10 )|(np.count_nonzero(Y_training == -1) < 10 )):
        permutation = np.random.permutation(m)
        m = dataset.shape[0]
        permutation = np.random.permutation(m) # random permutation
        X = X[permutation]
        Y = Y[permutation]
        X_training = X[:m_training]
        Y_training = Y[:m_training]
        
print(".................................................")
if (np.count_nonzero(Y_training == -1) >= 10 ) & (np.count_nonzero(Y_training == 1) >= 10 ) :
    print("Now the training set is ok!")
#X_test = instances for test set
X_test = X[m_training::]
#Y_test = labels for the test set
Y_test = Y[m_training::]

print(Y_training) #to make sure that Y_training contains both 1 and -1
print(m_test)

print("Shape of training set: " + str(X_training.shape))
print("Shape of test set: " + str(X_test.shape)) 


The training set contains at least 10 elements of class 1 AND at least 10 elements of class -1
.................................................
Now the training set is ok!
[-1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1  1  1  1  1 -1 -1
  1  1 -1 -1  1 -1 -1  1  1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1  1 -1  1
  1 -1  1  1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1
  1 -1  1  1  1  1  1  1  1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  1  1  1 -1
  1  1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1  1  1  1  1 -1  1  1 -1 -1 -1  1
 -1 -1 -1 -1 -1  1  1  1  1  1 -1 -1  1 -1  1  1  1 -1 -1  1 -1  1 -1  1
  1 -1 -1  1  1  1 -1  1  1  1 -1 -1 -1 -1  1  1 -1 -1 -1  1  1 -1  1 -1
  1 -1 -1 -1 -1  1 -1  1  1  1  1 -1 -1  1]
78
Shape of training set: (182, 3)
Shape of test set: (78, 3)


In [12]:
#add a 1 to each sample (homogeneous coordinates)
X_training = np.hstack((np.ones((m_training,1)),X_training)) 
X_test = np.hstack((np.ones((m_test,1)),X_test))

In [13]:
def perceptron(X, Y, max_num_iterations):
    
    #initialize
    best_w = np.zeros((np.size(X,1)))
    temp_w = np.zeros((np.size(X,1)))
    array_errs = []
    error = 0
    iterations = 0
    
    #check for misclassified with initialization values, can be avoided
    #since all weights are 0's and whole training set will be considered as misclassified
    for k in range(m_training):
        if ( (Y[k]*np.dot(X[k],best_w) ) <= 0 ):
            array_errs.append(k)
    best_error = len(array_errs)/m_training
    
    #for now best_w is all zeros
    #and best_errs are misclassified examples using it  
    #real perceptron
    for iterations in range(max_num_iterations):
        index = np.random.choice(array_errs) #draw from misclassified ones
        temp_w += Y[index]*X[index]          #then update the temp_w (1 iteration)
        array_errs = []
        
        for k in range(m_training): #check for errors in whole training dataset with sol
            if ( (Y[k]*np.dot(X[k],temp_w)) <= 0 ): #this is a missclassification
                array_errs.append(k)
        error = len(array_errs)/m_training
        
        if (error == 0): 
            print("Optimal solution found!")
            best_w = temp_w 
            best_error = error
            break
            
        if (error < best_error):
            best_w = temp_w #if it is smaller then update best_w and best_err
            best_error = error
    
    return best_w, best_error

In [6]:
# A template is provided, but feel free to build a different implementation

def perceptron(X, Y, max_num_iterations):
    # Place in this function the main section of the perceptron algorithm
    
    #init the algorith with w=0, use a best_w variable to keep track of the best solution
    curr_w = np.zeros((np.size(X,1))) # initialize to zeros
    best_w = np.zeros((np.size(X,1))) # initialize to zeros
 
    best_error = 0
    error = 0
    num_iter = 0
    misclassified = []
    

    num_misclassified = len(misclassified)
    index_misclassified = range(num_misclassified+1)
    
    for i in range(m_training):
        if ( (Y[i]*np.dot(X[i],best_w) ) <= 0 ):
             misclassified.append(i)
    best_error = num_misclassified/m_training
    
    while ((index_misclassified != -1) and (num_iter < max_num_iterations)):
        
        index_misclassified = -1
        num_misclassified = 0
        index_misclassified  = np.random.choice(misclassified)
        # avoid working always on the same sample, 
        # you can use a random permutation or randomize the choice of misclassified
        
        curr_w += Y[index_misclassified]*X[index_misclassified] 
        misclassified = []
        
        for i in range(m_training):
            if ((Y[i]*np.dot(X[i],curr_w)) <= 0 ): 
                misclassified.append(i)
        error = len(misclassified)/m_training
        
        if (error == 0): 
            print("The optimal solution is found")
            best_w = curr_w 
            best_error = error
            break  # go out of the cicle
            
        if (error < best_error):
            best_w = curr_w 
            best_error = error
            
    return best_w, best_error

In [14]:
#now run the perceptron for 100 iterations
w_found, error = perceptron(X_training,Y_training, 1)
print("Best solution: " , w_found)
print("Training Error of perpceptron (100 iterations): " + str(error))

Best solution:  [  1. 213. 114.  25.]
Training Error of perpceptron (100 iterations): 0.4945054945054945
